## 2019

In [2]:
import pandas as pd
import numpy as np

df2019 = pd.read_excel('./data/2019.xlsx', sheet_name= 'Sheet1', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=55)
df2019.columns = ['팀 코드', '주제명', '팀원1', '팀원2', '팀원3']

df2019.head()

,팀 코드,주제명,팀원1,팀원2,팀원3
0,가-1,어떻게 해수욕장 인구를 효율적으로 측정할까?,2115 박장현,2117 송지원,NaN
1,가-2,"해수욕장, 언제 갈래요?",2401 곽서현,2402 박지윤,NaN
2,가-3,이안류 위험도를 예측하여 대비할 수 있을까?,2403 이영진,2503 박소예,NaN
3,가-4,어떻게 낚시 이용자를 증대시키고 해얀지역 경제를 활성화시킬 수 있을까?,2506 진혜민,2510 노윤호,NaN
4,가-5,해수욕장을 100%로 즐기려면?,2208 김도훈,2314 이정홍,NaN


In [3]:
df2019['팀 코드'] = df2019['팀 코드'].str.replace('가', 'A')
df2019['팀 코드'] = df2019['팀 코드'].str.replace('나', 'B')
df2019['팀 코드'] = df2019['팀 코드'].str.replace('다', 'C')
df2019['팀 코드'] = df2019['팀 코드'].str.replace('라', 'D')

df2019['팀 코드'] = df2019['팀 코드'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))

df2019['팀원1학번'], df2019['팀원1이름'] = df2019['팀원1'].apply(lambda x: x[:4] if x is not np.nan else x), df2019['팀원1'].apply(lambda x: x[4:] if x is not np.nan else x)

df2019['팀원2학번'], df2019['팀원2이름'] = df2019['팀원2'].apply(lambda x: x[:4] if x is not np.nan else x), df2019['팀원2'].apply(lambda x: x[4:] if x is not np.nan else x)

df2019['팀원3학번'], df2019['팀원3이름'] = df2019['팀원3'].apply(lambda x: x[:4] if x is not np.nan else x), df2019['팀원3'].apply(lambda x: x[4:] if x is not np.nan else x)
df2019.drop(['팀원1', '팀원2', '팀원3'], axis=1, inplace= True)

df2019.tail()

,팀 코드,주제명,팀원1학번,팀원1이름,팀원2학번,팀원2이름,팀원3학번,팀원3이름
48,D08,어선의 최적항로는 어떤 모양일까?,3408,김장현,3409,김재영,NaN,NaN
49,D09,소형 배로 적조 문제를 해결하고 해양데이터까지 수집하면 어떨까?,3416,서영국,3504,공명준,NaN,NaN
50,D10,뱃멀미를 어떻게 예방할 수 있을까?,1110,배시우,1216,이상민,1518,조성현
51,D11,해류와 암초의 영향을 고려한 선박의 최적 경로는?,1108,박정원,1109,박주영,1415,이유신
52,D12,최적의 선박 항로는? 골든 타임 확보를 위한 선박의 안전성 보장,1406,김원준,1419,최강훈,NaN,NaN


In [4]:
properties = ['팀 코드', '주제명', '팀원1 학번', '팀원1 이름', '팀원2 학번', '팀원2 이름', '팀원3 학번', '팀원3 이름']

def get_student_number(student_number):
  try:
    return str(int(student_number))
  except:
    return '0000'

def set_metadata(team_info, document, year = 2019):
  document.metadata = {}
  team_info.fillna('0')

  document.metadata['Year'] = str(year)
  document.metadata['Team code'] = str(team_info['팀 코드'])
  document.metadata['Title'] = str(team_info['주제명'])

  try:
    document.metadata['Team name'] = str(team_info['팀명'])
  except:
    document.metadata['Team name'] = '0'

  document.metadata['Teammate #1 name'] = str(team_info['팀원1이름'])
  document.metadata['Teammate #1 number'] = get_student_number(team_info['팀원1학번'])
  document.metadata['Teammate #2 name'] = str(team_info['팀원2이름'])
  document.metadata['Teammate #2 number'] = get_student_number(team_info['팀원2학번'])
  document.metadata['Teammate #3 name'] = str(team_info['팀원3이름'])
  document.metadata['Teammate #3 number'] = get_student_number(team_info['팀원3학번'])
  document.metadata['Physics'] = 'False'
  document.metadata['Chemistry'] = 'False'
  document.metadata['Biology'] = 'False'
  document.metadata['EarthScience'] = 'False'
  
  
  try:
    document.metadata['Youtube link'] = str(team_info['YOUTUBE 영상 주소'])
  except:
    document.metadata['Youtube link'] = '0'

  return document




In [5]:
import os
from langchain_community.document_loaders import PyMuPDFLoader

import re	# 정규식 라이브러리
p = re.compile('\d+') # 괄호 안에 있는 학번 추출

PDF_DIR = 'OceanICT_Data/2019_OceanICT/포스터/'
files = os.listdir(PDF_DIR)
file_list = sorted(files, key = lambda x: int(p.match(x).group()))

split_documents19 = []

for f in file_list:
  loader = PyMuPDFLoader(PDF_DIR + f)
  document = loader.load()[0]
  number = p.findall(f)[1]

  for i in range(1,4):
    result = df2019.loc[df2019[f'팀원{i}학번'] == number]
    if not result.empty:
      break

  try:
    new_document = set_metadata(result.iloc[0], document)
    split_documents19.append(new_document)
  except:
    continue

In [6]:
print(len(split_documents19))

54


In [7]:
from pprint import pprint

type(split_documents19[0])

langchain_core.documents.base.Document

## 2020

In [8]:
df20 = pd.read_excel('./data/2020.xlsx', sheet_name= '팀 코드 정리', usecols=[i + 8 for i in range(7)], skiprows=3, nrows=72)
df20.drop(['유형'], axis=1, inplace=True)
df20 = df20.rename(columns={'연구 제목': '주제명'})
df20.tail()

,팀원1,팀원2,팀원3,주제명,팀 번호,YOUTUBE 영상 주소
66,2406 김근형,2409 김준우,NaN,효율적인 등대 배치를 위한 통계적인 최적화 방법에 대한 탐구,D-8,https://youtu.be/yXW9bSgih-Y
67,2407 김민성,2412 류운비,NaN,연안에서 최적의 선박 회피경로 구하기,D-9,https://youtu.be/7qKrCrHhFQc
68,2520 최호영,NaN,NaN,선박 간 정보공유 플랫폼 구축을 통한 어획의 편의성 증대,D-10,https://youtu.be/SB1BPdxQ-h4
69,3214 정상,3314 한경찬,NaN,선박에서 코로나바이러스 피해를 줄일 수 있을까?,D-11,https://youtu.be/sV8qrHp3vcs
70,3307 박관우,3407 박성현,NaN,선박 사고 시 가장 효율적인 대처 방안 모색,D-12,https://youtu.be/-tG4NkGq__4


In [9]:
df20['팀 코드'] = df20['팀 번호'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))

for i in range(1, 4):
    df20[f'팀원{i}학번'], df20[f'팀원{i}이름'] = df20[f'팀원{i}'].apply(lambda x: x[:4] if x is not np.nan else x), df20[f'팀원{i}'].apply(lambda x: x[5:] if x is not np.nan else x)

df20.drop(['팀 번호', '팀원1', '팀원2', '팀원3'], axis=1, inplace= True)

df20.head()

,주제명,YOUTUBE 영상 주소,팀 코드,팀원1학번,팀원1이름,팀원2학번,팀원2이름,팀원3학번,팀원3이름
0,해운대 해수욕장 파속측정을 통한 이안류 발생시 알림 수신 프로그래밍,https://www.youtube.com/watch?v=JSnm4dtmU5c,A01,1118,이현승,1115,이영진,1308,김정현
1,해양데이터를 이용해 이안류 피해를 줄일 수 있을까?,https://www.youtube.com/watch?v=Y5YvaB7awd4,A02,1210,안동준,1212,이건희,1215,장연수
2,해양 양식장 관리 시스템 설계,https://youtu.be/En4Bn1UEuyw,A03,1216,정민서,1418,장문정,1515,이승헌
3,수온에 따른 어획량예측,https://youtu.be/eNqs5N7W9v8,A04,1311,박정윤,1312,백재승,1320,정윤상
4,해수욕장 인명사고 예방을 위한 경보 시스템 제작,https://www.youtube.com/watch?v=m4bCSDfgRaA,A05,2103,안주연,2104,이수민,NaN,NaN


In [10]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
import tqdm

import re	# 정규식 라이브러리
p = re.compile('\d+') # 괄호 안에 있는 팀코드 추출

DIR = 'OceanICT_Data/2020_OceanICT/'
file_list = os.listdir(DIR)
file_list_pdf = [file for file in file_list if file.endswith(".pdf")]
file_list_pptx = [file for file in file_list if file.endswith(".pptx")]


split_documents20 = []

for f in file_list_pdf:
  loader = PyMuPDFLoader(DIR + f)
  document = loader.load()[0]
  number = p.findall(f)[1]

  for i in range(1,4):
    result = df20.loc[df20[f'팀원{i}학번'] == number]
    if not result.empty:
      break

  new_document = set_metadata(result.iloc[0], document, 2020)
  split_documents20.append(new_document)

for f in tqdm.tqdm(file_list_pptx):
  loader = UnstructuredPowerPointLoader(DIR + f)
  document = loader.load()[0]
  number = p.findall(f)[1]

  for i in range(1,4):
    result = df20.loc[df20[f'팀원{i}학번'] == number]
    if not result.empty:
      break

  new_document = set_metadata(result.iloc[0], document, 2020)
  split_documents20.append(new_document)

100%|██████████| 70/70 [00:08<00:00,  8.18it/s]


In [11]:
len(split_documents20)

74

## 2021

In [12]:
df21_A = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 문화와 관광 진흥', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=20)
df21_B = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 생태계 및 환경 보존', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=30)
df21_C = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 자원의 이용 기반 구축', usecols=[i + 2 for i in range(5)], skiprows=2, nrows=34)
df21_D = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 선박 관련 기술', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=10)

df21 = pd.concat([df21_A, df21_B, df21_C, df21_D])
df21 = df21.rename(columns={'주제': '주제명'})
df21.head()

,연번,주제명,팀원1,팀원2,팀원3
0,A-1,해변 관광환경 개선 및 만족도 향상을 위한 해변 추천 서비스 제작하기,1413 신재현,1420 하승훈,NaN
1,A-2,해양 정보를 이용한 플레이리스트 추천 프로그램,2104 윤현서,2105 이다은,NaN
2,A-3,인공지능과 오픈 API를 이용한 해양 경보 어플리케이션 제작,2108 김경태,2119 최준영,NaN
3,A-4,부산 수산시장의 해산물 가격 판단 서비스,2213 안동준,2215 장연수,NaN
4,A-5,역기압 효과를 이용한 파고 높이 계산을 통해 어떻게 풍랑주의보 알리미 제작할 수 있을까?,2217 정재민,2218 조현준,NaN


In [13]:
df21['팀 코드'] = df21['연번'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))
df21['연번'] = df21['연번'].str.replace('-', '')

for i in range(1, 4):
    df21[f'팀원{i}학번'], df21[f'팀원{i}이름'] = df21[f'팀원{i}'].apply(lambda x: x[:4] if x is not np.nan else x), df21[f'팀원{i}'].apply(lambda x: x[5:] if x is not np.nan else x)

df21.drop(['팀원1', '팀원2', '팀원3'], axis=1, inplace= True)

df21.head()

,연번,주제명,팀 코드,팀원1학번,팀원1이름,팀원2학번,팀원2이름,팀원3학번,팀원3이름
0,A1,해변 관광환경 개선 및 만족도 향상을 위한 해변 추천 서비스 제작하기,A01,1413,신재현,1420,하승훈,NaN,NaN
1,A2,해양 정보를 이용한 플레이리스트 추천 프로그램,A02,2104,윤현서,2105,이다은,NaN,NaN
2,A3,인공지능과 오픈 API를 이용한 해양 경보 어플리케이션 제작,A03,2108,김경태,2119,최준영,NaN,NaN
3,A4,부산 수산시장의 해산물 가격 판단 서비스,A04,2213,안동준,2215,장연수,NaN,NaN
4,A5,역기압 효과를 이용한 파고 높이 계산을 통해 어떻게 풍랑주의보 알리미 제작할 수 있을까?,A05,2217,정재민,2218,조현준,NaN,NaN


In [14]:
import os
from langchain_community.document_loaders import PyMuPDFLoader

import re	# 정규식 라이브러리

PDF_DIR = 'OceanICT_Data/2021_OceanICT/포스터/'
file_list = os.listdir(PDF_DIR)

split_documents21 = []
a= []

for f in file_list:
  loader = PyMuPDFLoader(PDF_DIR + f)
  try:
    document = loader.load()[0]
  except:
    continue
  result = df21.loc[df21[f'연번'] == f[:-4]]

  new_document = set_metadata(result.iloc[0], document, year = 2021)
  a.append(result.iloc[0])
  split_documents21.append(new_document)

In [15]:
split_documents21

[Document(metadata={'Year': '2021', 'Team code': 'A02', 'Title': '해양 정보를 이용한 플레이리스트 추천 프로그램', 'Team name': '0', 'Teammate #1 name': '윤현서', 'Teammate #1 number': '2104', 'Teammate #2 name': '이다은', 'Teammate #2 number': '2105', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0000', 'Physics': 'False', 'Chemistry': 'False', 'Biology': 'False', 'EarthScience': 'False', 'Youtube link': '0'}, page_content="Busan science high school\n2021 Ocean ICT Festival\n2021 BOIF\nYoutube 영상QR\nA \n2\n멜론DJ 플레이리스트\n크롤링\n많이겹치는곡부터나열해서새로운플\n레이리스트구성\n기상청해양정보크롤링\n기상청해양정보이용\n해당지역의습도, 온도, 풍속을기준으\n로날씨분류\n해양오염이심각해지고있는오늘날, 부산의트레이드마크인바다\n를지키기위해그날날씨에맞는음악을멜론차트에서추천하면\n서기상청의해양정보를첨부하여사람들의해양오염에대한경각\n심을높이고자하였다. 우리는접근성이좋은날씨별음악으로플\n레이리스트를구성해보았다.\n탐구목적\n탐구동기\n융합분야\n어느날멜론월간이용자수가852만명에달한다는기사를접하\n고, 많은사람들이이용하는음악어플에해양정보를나타낸다면,\n해양오염에대한경각심과함께해양오염에많은사람들이관심을\n가질것이라고생각했다.\n알고리즘설명\n1. 데이터수집\n- 기상청날씨누리바다관측자료에서실시간으로바다정보를크\n롤링한다.\n- 가장인기있는멜론DJ 플레이리스트10개를크롤링한후많이\n겹치는상위50개의곡을추린다. 그리고멜론사이트에추린곡\n으로구성된플레이리스트를업로드한다.\n2. 날씨에따른데이터출력\n- 각습도, 온도

In [16]:
import pandas as pd

team_data = './data/2023.csv'

df = pd.read_csv(team_data)
df.head()

,팀 코드,주제명,해양 관련 분야 선택,분야,팀원1 학번,팀원1 이름,팀원2 학번,팀원2 이름,팀원3 학번,팀원3 이름,기타 과목,물리학,화학,생명과학,지구과학,유튜브 링크
0,A01,아두이노와 수위조절센서를 이용한 안전 부표 제작,해양 문화와 관광 진흥,A,1418,임성훈,1406,이서경,NaN,NaN,NaN,v,NaN,NaN,v,https://youtube.com/watch?v=8TJ7Fig9mOY&featur...
1,A02,"거리와 물때, 조수간만의 차, 해양 오염 정도를 고려한 갯벌 추천",해양 문화와 관광 진흥,A,2304,이지원,2308,권민찬,NaN,NaN,NaN,NaN,NaN,V,V,https://youtu.be/6CeiASdQX94
2,A03,ACO algorithm을 이용해 2030 엑스포를 위한 ​해상 도시 계획의 교통문...,해양 문화와 관광 진흥,A,2311,김민수,2312,김채준,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://youtu.be/5B3b9EZyLdU
3,A04,기후위기와 지역별 해적의 상관관계,해양 문화와 관광 진흥,A,2313,도영운,2314,신승창,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/watch?v=gAL6a0S56UM&t=3
4,A05,인공지능아~ 해운대 까페에서 먹을 음료 알려줘,해양 문화와 관광 진흥,A,2318,정민영,2319,정보권,NaN,NaN,V(관광 서비스),NaN,NaN,NaN,NaN,https://youtu.be/7M3h2ApRLB4


In [17]:
# 단계 1: 문서 로드(Load Documents)
# 단계 2: 문서 분할(Split Documents) -> 스킵 (사유: 포스터 섞이더라)

import os
import tqdm
from langchain_community.document_loaders import PyMuPDFLoader

import re	# 정규식 라이브러리
p = re.compile('\(([^)]+)') # 괄호 안에 있는 팀코드 추출

PDF_DIR = './data/pdfs/2023/'
file_list = os.listdir(PDF_DIR)
split_documents23 = []

for f in tqdm.tqdm(file_list):
  loader = PyMuPDFLoader(PDF_DIR + f)
  document = loader.load()[0]

  team_code = p.findall(f)[0]

  result = df.loc[df['팀 코드'] == team_code]
  result.columns = ['팀 코드', '주제명', '1', '2', '팀원1학번', '팀원1이름', '팀원2학번', '팀원2이름', '팀원3학번', '팀원3이름', 'etc', 'Physics', 'Chemistry', 'Biology', 'EarthScience', 'YOUTUBE 영상 주소']

  subjectnames = ['Physics', 'Chemistry', 'Biology', 'EarthScience']

  subjects = []

  for i in subjectnames:
    if list(result[i])[0] in ['v', 'V']:
      subjects.append(i)


  try:
    new_document = set_metadata(result.iloc[0], document, '2023')
    for i in subjects:
      new_document.metadata[i] = 'True'
    split_documents23.append(new_document)
  except:
    continue
    #split_documents.append(new_document)

  0%|          | 0/92 [00:00<?, ?it/s]

  3%|▎         | 3/92 [00:00<00:03, 26.27it/s]

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type



100%|██████████| 92/92 [00:03<00:00, 23.86it/s]


In [18]:
len(split_documents23)

92

In [19]:
split_documents23

[Document(metadata={'Year': '2023', 'Team code': 'B42', 'Title': '옥시벤존 농도 모니터링 프로그램을 통한 해양생태계 보호', 'Team name': '0', 'Teammate #1 name': '조수현', 'Teammate #1 number': '3203', 'Teammate #2 name': '홍수민', 'Teammate #2 number': '3306', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0000', 'Physics': 'False', 'Chemistry': 'True', 'Biology': 'True', 'EarthScience': 'False', 'Youtube link': 'https://youtu.be/LX37iXYXHTA'}, page_content='Busan science high school\n2023 Ocean ICT Festival\n2023 BOIF\nYoutube 영상 QR\nB \n42\nQR 코드 영역\nQR 삽입 후 \n테두리 삭제\n옥시벤존농도모니터링프로그램을\n통한해양생태계보호\n조수밤바다홍수경보\n3203 조수현 3306 홍수민\n1. 탐구 목적\n[2022 Ocean ICT]\n해수의 옥시벤존 농도를 측정해 특정 값 이상이 되면 경고를 울리고 옥시벤존 분해 물질을 투여하는 코드를 작성하여 옥시벤존의 농도를 줄이는 탐구를 진행함.\n[2023 Ocean ICT]\n하지만, 해수의 옥시벤존을 직접적으로 분해시키는 방법은 매우 일시적인 효과를 가지며, 가성비와 효율이 좋지 못하다는 단점이 있었다.\n따라서, 이번에는 접근법에 변화를 주어 바다로 가는 사람들이나 해변에서 활동하는 사람들을 대상으로 옥시벤존과 옥티노세이트가 함유된 선크림을 사용하지\n않도록 경고하거나 대체 제품을 추천하는 프로그램을 제작하고자 하였다.\n[해양 문제]\n선크림의 주성분(옥시벤존, 옥티노세이트) :\n산호의DNA 손상유도해, 어린 산호의 기형 생

## 2024

In [20]:
df24 = pd.read_excel('./data/2024.xlsx', sheet_name= '팀별', usecols=[i + 1 for i in range(9)], skiprows=2, nrows=98)

df24 = df24.rename(columns={'섹션\n번호': '팀 코드'})
df24.drop(0, inplace=True)
df24.drop(['학년'], axis= 1,inplace=True)

df24['팀 코드'] = df24['팀 코드'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))
df24.fillna('0000', inplace=True)

for i in range(1, 4):
    try:
        df24[f'팀원{i}학번'] = df24[f'팀원{i} 학번'].astype(int).astype(str)
        df24.rename(columns={f'팀원{i} 이름': f'팀원{i}이름'},inplace=True)
        df24.drop([f'팀원{i} 학번'],inplace=True,axis=1)
    except:
        continue

df24['주제명'] = df24['팀명']
df24['과목'] = ['' for i in range(df24.shape[0])]
df24['얘 들어갔어요'] = np.zeros(df24.shape[0],dtype=int)

In [21]:
df24_1 = pd.read_excel('./data/2024_1.xlsx', sheet_name = '팀별',usecols=[i + 2 for i in range(14)], skiprows=2, nrows=98)

df24_1.drop(0, inplace=True)
df24_1.drop(df24_1.columns[6:8],axis=1,inplace=True)
df24_1.columns = ['팀원1 학번','팀원1 이름', '팀원2 학번','팀원2 이름','팀원3 학번','팀원3 이름','물','화','생','지','기타', '주제명']

df24_1.head(10)

,팀원1 학번,팀원1 이름,팀원2 학번,팀원2 이름,팀원3 학번,팀원3 이름,물,화,생,지,기타,주제명
1,1102.0,방윤서,1306.0,정지민,NaN,NaN,NaN,1,NaN,NaN,NaN,난류와 ph에 의한 갯녹음화지도 만들기
2,1103.0,이서현,1104.0,이채연,NaN,NaN,1,NaN,NaN,NaN,NaN,스마트 유령어구 위치 추적 시스템
3,1105.0,최연정,1106.0,하윤아,NaN,NaN,1,1,NaN,NaN,NaN,폐수 유출 지점 탐색
4,1110.0,박관호,1112.0,박준성,NaN,NaN,1,NaN,NaN,1,NaN,이안류와 영향인자의 상관관계 분석을 통한 이안류 예측 및 대응 알고리즘
5,1113.0,송민호,1116.0,오진우,NaN,NaN,NaN,NaN,1,1,NaN,멸종위기 해양생물 개체수와 서식지역 파악하기
6,1114.0,송범준,1118.0,이정훈,NaN,NaN,NaN,NaN,1,NaN,NaN,딥러닝 기반 멸종 위기 어류 판별 및 보호
7,1117.0,이걸묵,1409.0,김기환,NaN,NaN,1,NaN,NaN,NaN,NaN,전자기유도를 이용한 파도 분석 장치 및 고효율 발전기 고안
8,1203.0,장서인,1214.0,정도현,NaN,NaN,NaN,NaN,NaN,1,NaN,위도별 해수의 온도와 해수 온도 상승 위험 정도 알림
9,1205.0,공준호,1211.0,이민찬,NaN,NaN,1,NaN,NaN,1,NaN,방사능 오염수의 확산에 따른 악영향을 최소화하기 위한 방법 찾기
10,1206.0,김민준,1213.0,장현제,NaN,NaN,1,NaN,NaN,NaN,NaN,모듈형 쌍동선의 자동 결합을 위한 코드 개발


In [22]:
subject_name = ['Physics', 'Chemistry', 'Biology', 'EarthScience']

for i in range(df24_1.shape[0]):
    subjects = list(df24_1.loc[i + 1, ['물','화','생','지']])
    subject = []
    for j in range(4):
        if subjects[j] == 1:
            subject.append(subject_name[j])

    teamname = str(int(df24_1.iloc[i, 0]))
    projectname = df24_1.iloc[i, 11]

    
    df24.loc[df24[f'팀원1학번'] == teamname, '주제명'] = projectname
    df24.loc[df24[f'팀원1학번'] == teamname, '과목'] = ','.join(subject)
    for j in range(3):
        print(df24_1.iloc[i, 2*j + 1])
        df24.loc[df24[f'팀원1학번'] == teamname, f'팀원{j + 1}이름'] = df24_1.iloc[i, 2*j + 1]

df24.head(10)

방윤서
정지민
nan
이서현
이채연
nan
최연정
하윤아
nan
박관호
박준성
nan
송민호
오진우
nan
송범준
이정훈
nan
이걸묵
김기환
nan
장서인
정도현
nan
공준호
이민찬
nan
김민준
장현제
nan
신승환
정현철
nan
양승우
최문기
nan
정재현
정휘찬
nan
봉가현
정세연
nan
정은수
정지민
nan
정은수
김건영
nan
권오준
이지우
nan
김창욱
허윤
nan
노현우
최승기
nan
박예일
이재원
nan
권나경
김수현
nan
김미소
이희원
nan
장소형
장윤
nan
강준호
이찬서
nan
구은현
성민찬
nan
김건영
김정원
nan
김도훈
최신호
nan
최선웅
한서준
nan
강혜진
이인규
nan
김소원
박지유
nan
권민근
서정안
nan
김건우
박상진
정수환
김범수
황우진
nan
선우무경
안상민
nan
김지우
이재영
nan
배시아
이윤승
nan
정여진
이재호
nan
고민재
김성민
nan
김상우
박재성
nan
박준영
박찬휘
백창훈
안병훈
이승후
nan
이건우
백종윤
nan
이성민
장동주
nan
강지윤
여유진
nan
김서경
이나원
nan
전예원
박주형
nan
김동건
임건택
nan
김바다
정지훈
nan
김지홍
양성수
nan
류장현
표정훈
nan
박준서
이원혁
nan
이동윤
최현민
nan
구승현
김민서
nan
김아윤
박지환
nan
정유진
하은결
nan
김민기
김환
nan
김석현
박시형
nan
김성윤
유민석
nan
김태규
박예준
nan
이승준
김은우
nan
이언석
정우진
홍진수
김유빈
맹형주
nan
서해원
손지민
nan
은채현
남희수
nan
한예송
류홍재
nan
김도현
김찬영
송준환
김태윤
이석준
nan
안승준
허석현
nan
유예준
임성훈
nan
정우성
조은우
nan
김민채
김윤정
nan
양수빈
유서윤
nan
이서경
강석현
nan
권민성
이우진
nan
권해정
김성민
nan
김민규
김준민
nan
김태희
이호준
nan
서재원
정희도
nan
조용석
하서준
nan
권지민
김민수
이지원
김정현
조민경
nan
윤성빈
이경록
nan
김채준
노현수
nan
신동원
이윤

,팀 코드,팀원1이름,팀원2이름,팀원3이름,팀명,팀원1학번,팀원2학번,팀원3학번,주제명,과목,얘 들어갔어요
1,B01,방윤서,정지민,NaN,클로렐라쌀밥,1102,1306,0,난류와 ph에 의한 갯녹음화지도 만들기,Chemistry,0
2,B02,이서현,이채연,NaN,유령인간,1103,1104,0,스마트 유령어구 위치 추적 시스템,Physics,0
3,B03,최연정,하윤아,NaN,디버깅 마스터,1105,1106,0,폐수 유출 지점 탐색,"Physics,Chemistry",0
4,A01,박관호,박준성,NaN,정보통신기술,1110,1112,0,이안류와 영향인자의 상관관계 분석을 통한 이안류 예측 및 대응 알고리즘,"Physics,EarthScience",0
5,B04,0000,0000,0000,별미,1111,1505,0,별미,,0
6,B05,송민호,오진우,NaN,오진우뭐먹어나도줘,1113,1116,0,멸종위기 해양생물 개체수와 서식지역 파악하기,"Biology,EarthScience",0
7,B06,송범준,이정훈,NaN,바다의 왕자,1114,1118,0,딥러닝 기반 멸종 위기 어류 판별 및 보호,Biology,0
8,D01,이걸묵,김기환,NaN,바다와 고딩,1117,1409,0,전자기유도를 이용한 파도 분석 장치 및 고효율 발전기 고안,Physics,0
9,B07,장서인,정도현,NaN,SeaPerature,1203,1214,0,위도별 해수의 온도와 해수 온도 상승 위험 정도 알림,EarthScience,0
10,B08,공준호,이민찬,NaN,Radioactive BSS,1205,1211,0,방사능 오염수의 확산에 따른 악영향을 최소화하기 위한 방법 찾기,"Physics,EarthScience",0


In [23]:
df24_2 = pd.read_excel('./data/2024_2.xlsx', sheet_name = '팀별',usecols=[i + 2 for i in range(14)], skiprows=3, nrows=96)

# #df24 = df24.rename(columns={'섹션\n번호': '팀 코드'})
df24_2.drop(df24_2.columns[1:6],axis=1,inplace=True)
df24_2.drop(df24_2.columns[2:],axis=1,inplace=True)
for i in range(df24_2.shape[0]):
    num = str(int(df24_2.iloc[i, 1]))
    youtubelink = df24_2.iloc[i, 0]
    df24.loc[df24[f'팀원1학번'] == num, 'YOUTUBE 영상 주소'] = youtubelink
    if df24.loc[df24[f'팀원1학번'] == num].empty:
        df24.loc[df24[f'팀원2학번'] == num, 'YOUTUBE 영상 주소'] = youtubelink

In [24]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
import tqdm

import re	# 정규식 라이브러리
p = re.compile('[A-D]\d{2}') # 괄호 안에 있는 팀코드 추출
p1 = re.compile('[가-힣]+')
num = re.compile('\d{4} ?[가-힣]{2,4}')

DIR = 'OceanICT_Data/2024_OceanICT/'
file_list = os.listdir(DIR)

split_documents24 = []

for f in file_list:
    print(DIR + f)
    files = os.listdir(DIR + f)
    for i in files:
        if i.endswith('.pdf'):
            loader = PyMuPDFLoader(DIR + f +'/' + i)
        
        elif i.endswith(('.pptx', '.ppt')):
            loader = UnstructuredPowerPointLoader(DIR + f +'/' + i)
        
        else: continue

        print(i)

        number = p.findall(f)
        print(number)

        document = loader.load()[0]

        result = df24.loc[df24[f'팀 코드'] == number[0]]
        if not result.empty:
            df24.loc[df24[f'팀 코드'] == number[0], '얘 들어갔어요'] = 1
            break
    try:
        Subjects = list(result['과목'])[0].split(',')
    except:
        continue
    
    try:
        new_document = set_metadata(result.iloc[0], document, year = 2024)
        for j in Subjects:
            new_document.metadata[j] = 'True'
        split_documents24.append(new_document)
    except:
        continue

OceanICT_Data/2024_OceanICT/(B07)SEAPERATURE_1203장서인_1214정도현
SEAPERTATURE_1203장서인_1214정도현.pptx
['B07']
OceanICT_Data/2024_OceanICT/(B05)오진우뭐먹어나도줘_1113송민호 1116오진우
오션아이시티 차트_1113 송민호_1116 오진우.pptx
['B05']
OceanICT_Data/2024_OceanICT/(B41)해양소음을줄이다니럭키비키잖앙_2507권민성_2514이우진
해양소음을줄이다니럭키비키잖앙_2507권민성_2514이우진.pptx
['B41']
OceanICT_Data/2024_OceanICT/(B03)디버깅 마스터_1105최연정_1106하윤아
차트_디버깅 마스터_1105최연정_1106하윤아.pdf
['B03']
OceanICT_Data/2024_OceanICT/(B26)박혁거세_2215박준서_2215이원혁
박혁거세_2212박준서_2215이원혁.pptx
['B26']
OceanICT_Data/2024_OceanICT/(B18) 2105_2107
(B18) 2105_2107.pptx
['B18']
OceanICT_Data/2024_OceanICT/(B09)정봉_1303 봉가현_1304 정세연
정봉_1303 봉가현_1304 정세연.pptx
['B09']
OceanICT_Data/2024_OceanICT/(B20)타도일_2113안병훈_2116이승후
(B20) 2113 안병훈, 2116 이승후 ppt.pptx
['B20']
OceanICT_Data/2024_OceanICT/(B49)3학년3반_3303양수민_3308박새빈_3313정성원
3학년3반_3303양수민_3308박새빈_3313정성원.pptx
['B49']
OceanICT_Data/2024_OceanICT/(B37)정품_2413안승준_2419허석현
정품_오션 포스터.pptx
['B37']
OceanICT_Data/2024_OceanICT/(B50)오션윙즈_3305김세현_3514최온재
오션윙즈_3305김세현

In [25]:
df24.iloc[88]

팀 코드                                                     A12
팀원1이름                                                    김열린
팀원2이름                                                    정민영
팀원3이름                                                    NaN
팀명                                SEY(Search Extreme & Yell)
팀원1학번                                                   3306
팀원2학번                                                   3412
팀원3학번                                                      0
주제명                                         든든한 나의 보디가드, 오션이
과목                                           Physics,Biology
얘 들어갔어요                                                    0
YOUTUBE 영상 주소    https://www.youtube.com/watch?v=4i9Y0u8i1p0
Name: 89, dtype: object

In [26]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

In [27]:
import pprint 

for i in split_documents24:
    print(i.metadata)

{'Year': '2024', 'Team code': 'B07', 'Title': '위도별 해수의 온도와 해수 온도 상승 위험 정도 알림', 'Team name': 'SeaPerature', 'Teammate #1 name': '장서인', 'Teammate #1 number': '1203', 'Teammate #2 name': '정도현', 'Teammate #2 number': '1214', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0', 'Physics': 'False', 'Chemistry': 'False', 'Biology': 'False', 'EarthScience': 'True', 'Youtube link': 'https://m.youtube.com/watch?v=0A6afC6yvJY'}
{'Year': '2024', 'Team code': 'B05', 'Title': '멸종위기 해양생물 개체수와 서식지역 파악하기', 'Team name': '오진우뭐먹어나도줘', 'Teammate #1 name': '송민호', 'Teammate #1 number': '1113', 'Teammate #2 name': '오진우', 'Teammate #2 number': '1116', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0', 'Physics': 'False', 'Chemistry': 'False', 'Biology': 'True', 'EarthScience': 'True', 'Youtube link': 'https://youtu.be/cuQ1bILptjw?si=s93SbFUMLrUdB6PU'}
{'Year': '2024', 'Team code': 'B41', 'Title': '해역별 소음오염 예측 및 위험도 분류, 소음오염 저해방법 탐구', 'Team name': '0000', 'Teammate #1 name': '권민성', 'Teammate #1 number': '25

In [28]:
split_documents23

[Document(metadata={'Year': '2023', 'Team code': 'B42', 'Title': '옥시벤존 농도 모니터링 프로그램을 통한 해양생태계 보호', 'Team name': '0', 'Teammate #1 name': '조수현', 'Teammate #1 number': '3203', 'Teammate #2 name': '홍수민', 'Teammate #2 number': '3306', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0000', 'Physics': 'False', 'Chemistry': 'True', 'Biology': 'True', 'EarthScience': 'False', 'Youtube link': 'https://youtu.be/LX37iXYXHTA'}, page_content='Busan science high school\n2023 Ocean ICT Festival\n2023 BOIF\nYoutube 영상 QR\nB \n42\nQR 코드 영역\nQR 삽입 후 \n테두리 삭제\n옥시벤존농도모니터링프로그램을\n통한해양생태계보호\n조수밤바다홍수경보\n3203 조수현 3306 홍수민\n1. 탐구 목적\n[2022 Ocean ICT]\n해수의 옥시벤존 농도를 측정해 특정 값 이상이 되면 경고를 울리고 옥시벤존 분해 물질을 투여하는 코드를 작성하여 옥시벤존의 농도를 줄이는 탐구를 진행함.\n[2023 Ocean ICT]\n하지만, 해수의 옥시벤존을 직접적으로 분해시키는 방법은 매우 일시적인 효과를 가지며, 가성비와 효율이 좋지 못하다는 단점이 있었다.\n따라서, 이번에는 접근법에 변화를 주어 바다로 가는 사람들이나 해변에서 활동하는 사람들을 대상으로 옥시벤존과 옥티노세이트가 함유된 선크림을 사용하지\n않도록 경고하거나 대체 제품을 추천하는 프로그램을 제작하고자 하였다.\n[해양 문제]\n선크림의 주성분(옥시벤존, 옥티노세이트) :\n산호의DNA 손상유도해, 어린 산호의 기형 생

In [49]:
a = []
for i in split_documents24:
    a.append(i.metadata['Team code'])

In [52]:
print(sorted(a))

['A01', 'A02', 'A03', 'A04', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20', 'B20', 'B21', 'B23', 'B24', 'B25', 'B26', 'B27', 'B27', 'B28', 'B29', 'B30', 'B31', 'B31', 'B32', 'B33', 'B34', 'B35', 'B36', 'B37', 'B38', 'B39', 'B40', 'B41', 'B42', 'B43', 'B44', 'B45', 'B46', 'B47', 'B48', 'B49', 'B50', 'B51', 'B52', 'B53', 'B54', 'B55', 'B56', 'B56', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14']


In [30]:
split_documents24

[Document(metadata={'Year': '2024', 'Team code': 'B07', 'Title': '위도별 해수의 온도와 해수 온도 상승 위험 정도 알림', 'Team name': 'SeaPerature', 'Teammate #1 name': '장서인', 'Teammate #1 number': '1203', 'Teammate #2 name': '정도현', 'Teammate #2 number': '1214', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0', 'Physics': 'False', 'Chemistry': 'False', 'Biology': 'False', 'EarthScience': 'True', 'Youtube link': 'https://m.youtube.com/watch?v=0A6afC6yvJY'}, page_content='Busan science high school\n\n2024 Ocean ICT Festival\n\n2024 BOIF\n\nB\n\n07\n\nYoutube 영상 QR\n\n위도별 해수의 온도와 해수 온도 상승 위험 정도 알림\n\nSEAPERATURE\n\n1203장서인, 1214정도현\n\n<탐구 동기 및 목적>\n\n이 프로젝트는 지구 온난화가 심각한 문제로 떠오르며 해수의 온도 상승은 인류 및 생태계에 미치는 영향이 커지고 있다. 이를 알리고 사람들에게 실제 상황을 시각적으로 제공하여 보다 적극적인 대응을 이끌어내는 것에 목표를 둔다.\n\n<전체 개요>\n\n이 프로젝트는 정보과학과 지구과학을 융합하여 해수 온도의 변화를 시각화 하는 것이 목적이다. 지구과학에서의 이론과 데이터를 기반으로 해수 온도의 표준 및 변화 계산식을 구하고, 정보과학에서 그 식을 활용해 해수의 온도를 계산하고 그래프를 구현한다. 이를 통해 지구 온난화의 현재 상황과 위험성을 인지하고 대처할 수 있는 정보를 제공한다.\n\n<이론적 배경>\n\n위도별 표준 해수 온도(Sea Su

In [31]:
documents = []

for i in ['19', '20', '21', '23']:
    documents += globals()['split_documents' + i]

vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=OpenAIEmbeddings(),
        persist_directory=f'db/chroma_19to23_pdfs'
    )

vectorstore = Chroma.from_documents(
        documents=split_documents24,
        embedding=OpenAIEmbeddings(),
        persist_directory=f'db/chroma_2024_pdfs'
    )